# Tools for a Customer Outreach Campaign 💡

## Introduction

This notebook is designed to demonstrate the use of `tools in a customer outreach campaign using crewAI`. By the end of this notebook, you will have a clear understanding of how to create and manage agents, define their roles and tasks, and utilize tools to enhance their performance in a customer outreach campaign. The key elements of tools covered in this notebook are:
- Versatility
- Fault Tolerance
- Caching

**Note** If you're running this notebook on your own machine, you can install the following:
```Py
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

### Import libraries, APIs and LLM
- For more info, see [Serper](https://serper.dev)

In [ ]:
# Importing necessary classes from the crewai library
from crewai import Agent, Task, Crew

**Note** _Agent_: Represents an agent with specific roles and goals.      
         _Task_: Represents a task that the agent needs to perform.    
         _Crew_: Represents a group of agents working together on tasks.  

In [ ]:
# Import the required libraries
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

# Fetch the OpenAI API key using a utility function and store it in a variable
openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

# Set the environment variable for the OpenAI model name to 'gpt-4-turbo'
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

# Fetch the Serper API key using a utility function and set it as an environment variable
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## Creating Agents

- Agent 1 : the Agent class for a Sales Representative.  
This agent is responsible for identifying high-value leads that match the ideal customer profile. Besides, the agent is provided with a detailed backstory to guide its actions and decisions. The allow_delegation parameter is set to False, meaning this agent cannot delegate its tasks to other agents. The verbose parameter is set to True to enable detailed logging of the agent's actions.

In [ ]:
# Creating an instance of the Agent class for a Sales Representative.
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True
)

- Agent 2 : Creating an instance of the Agent class for a Lead Sales Representative.  
This agent is responsible for nurturing leads with personalized and compelling communications. The agent is provided with a detailed backstory to guide its actions and decisions. The allow_delegation parameter is set to False, meaning this agent cannot delegate its tasks to other agents. The verbose parameter is set to True to enable detailed logging of the agent's actions.

In [ ]:
# Creating an instance of the Agent class for a Lead Sales Representative.
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True
)

## Creating Tools

### crewAI Tools
crewAI Tools are specialized utilities designed to `enhance the functionality and performance of AI agents within the crewAI framework`. These tools provide agents with the capabilities to access and process various types of data, perform specific tasks, and interact with external resources efficiently. By equipping agents with versatile, fault-tolerant, and caching-enabled tools, crewAI ensures that agents can handle complex tasks and deliver high-quality results in customer outreach campaigns and other applications.

In [ ]:
# Import the required tools from the crewai_tools library
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [ ]:
# Creating instances of crewAI tools to be used by the agents.
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

**Note** The _DirectoryReadTool_ reads the contents of a specified directory. The _FileReadTool_ reads the contents of a specified file. The _SerperDevTool_ represents a custom tool for performing searches using the Serper API.

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [8]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

 Let's define a custom tool by subclassing BaseTool from crewAI_tools. This custom tool, SentimentAnalysisTool, is designed to analyze the sentiment of text. The tool has a name and a description to specify its purpose. The _run method is where the custom logic for sentiment analysis is implemented. For simplicity in this example, the tool always returns "positive" for any given text.

In [ ]:
# Defining a custom tool by subclassing BaseTool from crewAI_tools.
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [ ]:
# Creating an instance of the custom SentimentAnalysisTool.
sentiment_analysis_tool = SentimentAnalysisTool()

This instance will be used by the agents to analyze the sentiment of text, ensuring that communications are positive and engaging.

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [11]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [12]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [13]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	memory=True
)

## Running the Crew
Here, firstlly, let's define the input parameters for the crewAI agents to use during their tasks. These inputs provide specific details about the lead that the agents will profile and engage.The inputs include the lead's name, industry, key decision maker, position, and a recent milestone. Then let's kick off the crew with the defined inputs. This will initiate the tasks assigned to the agents, using the provided input parameters.

In [ ]:
# Defining the input parameters for the crewAI agents to use during their tasks.
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

# Kicking off the crew with the defined inputs.
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: To begin my analysis on DeepLearningAI, I need to gather comprehensive information about the company, including background, key personnel, recent business developments, and potential needs. I'll start by searching the internet for an overview of DeepLearningAI.

Action: Search the internet
Action Input: {"search_query": "DeepLearningAI company profile"} 


Search results: Title: Deeplearning.ai -

- Display the final result as Markdown.

In [15]:
from IPython.display import Markdown
Markdown(result)

Email Draft 1:
Subject: Elevating AI Education Together

Dear Andrew,

I hope this message finds you well. I've been closely following DeepLearningAI's impressive strides in transforming AI education, particularly your recent initiatives around generative AI and its applications in educational environments.

At CrewAI, we share your passion for harnessing AI to enhance learning experiences. Our advanced analytics tools are designed to provide educators and learners with personalized, impactful educational journeys. Given your recent course developments, I believe there's a fantastic opportunity for us to collaborate.

Would you be open to a discussion about how our analytics could integrate with your platforms to drive even greater learner engagement and success? I would love to share more about our success stories in educational technology.

Looking forward to the possibility of working together to shape the future of AI education.

Best regards,
[Your Name]
Lead Sales Representative, CrewAI

Email Draft 2:
Subject: Unlocking New Potentials in AI Education

Hello Andrew,

It's exciting to see DeepLearningAI at the forefront of discussions on AI in education, especially your approach to generative AI. At CrewAI, we've developed tools that not only complement such technologies but also enhance the overall efficacy of educational platforms.

I'm particularly interested in exploring how our generative AI tools can add value to your courses and initiatives. These tools have been instrumental in creating dynamic, interactive learning environments that can be tailored to individual needs.

Could we schedule a time to discuss potential integrations and explore synergies between our solutions? I believe there's much we can achieve together.

Warm regards,
[Your Name]
Lead Sales Representative, CrewAI

Email Draft 3:
Subject: Strengthening Community Engagement in AI Learning

Dear Andrew,

The community you've nurtured at DeepLearningAI is a testament to your commitment to accessible, high-quality AI education. At CrewAI, we recognize the power of community in the learning process and have developed tools that foster meaningful interactions and sustained engagement.

Our community engagement solutions could be a perfect match for DeepLearningAI, enhancing how learners connect, interact, and grow together. I'd love to dive deeper into your current strategies and discuss how our technology could support and expand your community initiatives.

Please let me know if you're available for a conversation. I'm eager to learn more about your vision and discuss how we can contribute to its realization.

Sincerely,
[Your Name]
Lead Sales Representative, CrewAI

These drafts aim to address the specific aspects of DeepLearningAI's recent activities and align CrewAI's offerings with their needs and goals, fostering a potential partnership.

**Explanation output** The output consists of three personalized email drafts aimed at engaging Andrew Ng, the CEO of DeepLearningAI, in a potential collaboration with CrewAI. Each draft is tailored to address different aspects of DeepLearningAI's recent activities and achievements while highlighting how CrewAI's tools and solutions can complement and enhance their efforts.

- Email Draft 1 focuses on the shared passion for AI education and suggests a discussion about integrating CrewAI's advanced analytics tools with DeepLearningAI's platforms to drive greater learner engagement and success.

- Email Draft 2 emphasizes the exciting developments in generative AI at DeepLearningAI and proposes exploring how CrewAI's generative AI tools can add value to their courses and initiatives, creating dynamic and interactive learning environments.

- Email Draft 3 highlights the importance of community engagement in AI learning and suggests that CrewAI's community engagement solutions could enhance how learners at DeepLearningAI connect, interact, and grow together.

The overall goal of these drafts is to align CrewAI's offerings with DeepLearningAI's needs and goals, fostering a potential partnership and collaboration.

## Conclusion 
This notebook demonstrated `how to use tools in a customer outreach campaign using crewAI`. By following the steps outlined, you created and managed agents, defined their roles and tasks, and utilized tools to enhance their performance. The agents successfully handled a customer outreach campaign, showcasing the effectiveness of the crewAI framework in identifying leads, profiling them, and crafting personalized outreach campaigns.